In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time 
from sqlalchemy import create_engine
import numpy as np

In [2]:
file_to_load = "list_of_stops.csv"
stops_data = pd.read_csv(file_to_load)
stops_data.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"


In [3]:
stops_df = stops_data[["STOP_ID", "STOP_NAME", "STATION_NAME", "STATION_DESCRIPTIVE_NAME", "MAP_ID", "Location"]]
stops_df

,STOP_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,Location
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,"(41.857908, -87.669147)"
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,"(41.857908, -87.669147)"
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,"(41.829353, -87.680622)"
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,"(41.829353, -87.680622)"
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,"(41.831677, -87.625826)"
5,30213,35-Bronzeville-IIT (Harlem-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,"(41.831677, -87.625826)"
6,30246,43rd (63rd-bound),43rd,43rd (Green Line),41270,"(41.816462, -87.619021)"
7,30245,43rd (Harlem-bound),43rd,43rd (Green Line),41270,"(41.816462, -87.619021)"
8,30025,51st (63rd-bound),51st,51st (Green Line),40130,"(41.80209, -87.618487)"
9,30024,51st (Harlem-bound),51st,51st (Green Line),40130,"(41.80209, -87.618487)"


In [4]:
stops_df[["Lat","Lon"]] = stops_df.Location.str.split(",", expand=True,)


stops_df.head()


/Users/thomasghormley/.conda/envs/pandas/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,STOP_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,Location,Lat,Lon
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,"(41.857908, -87.669147)",(41.857908,-87.669147)
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,"(41.857908, -87.669147)",(41.857908,-87.669147)
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,"(41.829353, -87.680622)",(41.829353,-87.680622)
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,"(41.829353, -87.680622)",(41.829353,-87.680622)
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,"(41.831677, -87.625826)",(41.831677,-87.625826)


In [5]:
stops_df = stops_df.drop(columns=['Location'])

stops_df.head()

,STOP_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,Lat,Lon
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,(41.857908,-87.669147)
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,(41.857908,-87.669147)
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,(41.829353,-87.680622)
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,(41.829353,-87.680622)
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,(41.831677,-87.625826)


In [6]:
stops_df['Lat'] = stops_df.Lat.str.replace("(", "")
stops_df['Lon'] = stops_df.Lon.str.replace(")", "")
stops_df = stops_df.rename(columns={"STOP_ID": "stop_id", "STOP_NAME": "stop_name", "STATION_NAME": "station_name",
                "STATION_DESCRIPTIVE_NAME": "station_descriptive_name", "MAP_ID": "map_id",
                "Lat": "lat", "Lon": "lon"})

In [7]:
stops_df.head()

,stop_id,stop_name,station_name,station_descriptive_name,map_id,lat,lon
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,41.857908,-87.669147
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,41.857908,-87.669147
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,41.829353,-87.680622
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,41.829353,-87.680622
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,41.831677,-87.625826


In [13]:
rds_connection_string = "postgres:taylormade@localhost:5433/CTA"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [14]:
stops_df.to_sql(name='stops_table', con=engine, if_exists='append', index=False)




In [15]:
pd.read_sql_query('select * from stops_table', con=engine).head()

,stop_id,stop_name,station_name,station_descriptive_name,map_id,lat,lon
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,41.857908,-87.669147
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,41.857908,-87.669147
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,41.829353,-87.680622
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,41.829353,-87.680622
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,41.831677,-87.625826
